# LLM Session Memory

Large Language Models are inherently stateless and have no knowledge of previous interactions with a user, or even of previous parts of the current conversation. While this may not be noticable when asking simple questions, it becomes a hinderance when engaging in long running conversations that rely on conversational context.

The solution to this problem is to append the previous conversation history to each subsequent call to the LLM.

This notebook will show how to use Redis to structure and store and retrieve this conversational session memory.

In [1]:
from typing import Dict, List
import cohere
import os

class CohereClient():
    def __init__(self, api_key: str = None, model: str = 'command-r-plus'):
        api_key = api_key or os.getenv("COHERE_API_KEY")
        self.client = cohere.Client(api_key)
        self._model = model

    def converse(self, prompt: str, context: List[Dict]) -> str:
        context = self.remap(context)
        response = self.client.chat(
                model=self._model,
                chat_history = context,
                message=prompt,
                )
        return response.text

    def remap(self, context) -> List[Dict]:
        ''' re-index the chat history to match the Cohere API requirements '''
        new_context = []
        for statement in context:
            if statement["role"] == "user":
                new_statement = {"role": "USER", "message": statement["content"]}
            elif statement["role"] == "llm":
                new_statement = {"role": "CHATBOT", "message": statement["content"]}
            elif statement["role"] == "system":
                new_statement = {"role": "SYSTEM", "message": statement["content"]}
            else:
                raise ValueError(f'Unknown chat role {statement["role"]}')
            new_context.append(new_statement)
        return new_context

In [2]:
from redisvl.extensions.session_manager import SemanticSessionManager
user_session = SemanticSessionManager(name='llm_chef', session_tag='123', user_tag='abc')
user_session.add_message({"role":"system", "content":"You are a helpful chef, assisting people in making delicious meals"})

client = CohereClient()

15:47:49 redisvl.index.index INFO   Index already exists, not overwriting.


On each call to the LLM we can first retrieve the recent history of the conversation. On the first call it will just be the preamble we set.

In [3]:
prompt = "can you give me some ideas for breakfast?"
context = user_session.get_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  can you give me some ideas for breakfast?

LLM:  Certainly! Here are some delicious breakfast ideas to start your day off right:

**1. Avocado Toast:**
   - Spread mashed avocado on a slice of toasted bread. Top it with a sprinkle of salt and pepper, a squeeze of lemon or lime juice, and a drizzle of extra virgin olive oil. You can also add sliced tomatoes, chopped onions, or a fried egg on top.

**2. Smoothie Bowl:**
   - Blend your favorite fruits (such as bananas, strawberries, blueberries, and mango) with a liquid base (like milk, yogurt, or juice) to create a thick and creamy smoothie. Pour it into a bowl and top it with granola, nuts, seeds, shredded coconut, or fresh fruit slices.

**3. Pancakes or Waffles:**
   - Whip up a batch of fluffy pancakes or waffles and serve them with a variety of toppings. Some options include fresh fruit, maple syrup, butter, whipped cream, chocolate chips, or a dusting of powdered sugar.

**4. Breakfast Burrito:**
   - Fill a flour tortilla 

Let's continue this conversation and inspect the context we're passing to the LLM as we go.

In [4]:
prompt = "can you give me the recipe for those pancakes?"
context = user_session.get_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  can you give me the recipe for those pancakes?

LLM:  Of course! Here is a classic recipe for fluffy and delicious pancakes:

**Ingredients:**

- 2 cups (250 grams) all-purpose flour
- 2 teaspoons baking powder
- 1 teaspoon salt
- 2 tablespoons sugar
- 2 cups milk
- 2 eggs
- 3 tablespoons melted butter, plus more for the pan
- 1 teaspoon vanilla extract (optional)

**Instructions:**

1. Whisk together the dry ingredients: In a large mixing bowl, combine the flour, baking powder, salt, and sugar. Whisk until well blended.

2. Combine the wet ingredients: In a separate mixing bowl, whisk together the milk, eggs, melted butter, and vanilla extract (if using). Make sure the melted butter has cooled down a bit so it doesn't cook the eggs.

3. Mix dry and wet ingredients: Slowly pour the wet ingredients into the dry ingredients while continuously whisking until you have a smooth batter. Be careful not to overmix; a few small lumps are okay.

4. Heat your griddle or pan: Place a nonsti

In [5]:
for ctx in context:
    print(ctx)

{'role': 'user', 'content': 'I changed my mind. Can you give me the first recipe from your list?'}
{'role': 'llm', 'content': 'Sure! Here is the recipe for "Easy One-Pot Pasta":\n\n**Ingredients:**\n- 2 cups (500 mL) of short pasta (such as penne, macaroni, or fusilli)\n- 1 tablespoon (15 mL) of olive oil\n- 1 onion, diced\n- 2 cloves of garlic, minced\n- 1 red or green bell pepper, diced\n- 1 can (28 ounces/796 mL) of diced tomatoes\n- 1 teaspoon (5 mL) of dried basil\n- 1 teaspoon (5 mL) of dried oregano\n- Salt and pepper to taste\n- 4 cups (1 L) of vegetable broth\n- 1 cup (250 mL) of grated mozzarella cheese (optional)\n\n**Instructions:**\n\n1. Heat the olive oil in a large pot over medium heat. Add the onion and garlic, and cook until softened, about 3-4 minutes.\n\n2. Add the bell pepper and cook for another 2-3 minutes.\n\n3. Stir in the diced tomatoes, basil, oregano, salt, and pepper. Let the mixture simmer for 5 minutes.\n\n4. Pour in the vegetable broth and bring the mixtu

In [6]:
prompt = "I am vegetarian. Can you remove the eggs?"
context = user_session.get_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I am vegetarian. Can you remove the eggs?

LLM:  Absolutely! You can make delicious vegetarian pancakes by omitting the eggs. Here's the modified recipe:

**Ingredients:**

- 2 cups (250 grams) all-purpose flour
- 2 teaspoons baking powder
- 1 teaspoon salt
- 2 tablespoons sugar
- 2 cups milk (dairy or plant-based alternative)
- 3 tablespoons melted butter, plus more for the pan
- 1 teaspoon vanilla extract (optional)

**Instructions:**

1. Whisk together the dry ingredients: In a large mixing bowl, combine the flour, baking powder, salt, and sugar. Whisk until well blended.

2. Combine the wet ingredients: In a separate mixing bowl, whisk together the milk, melted butter, and vanilla extract (if using). Make sure the melted butter has cooled down a bit so it doesn't curdle the milk.

3. Mix dry and wet ingredients: Slowly pour the wet ingredients into the dry ingredients while continuously whisking until you have a smooth batter. Again, don't overmix; a few small lumps are fine

In [7]:
for ctx in context:
    print(ctx)

{'role': 'system', 'content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': 'user', 'content': 'can you give me some ideas for breakfast?'}
{'role': 'llm', 'content': "Certainly! Here are some delicious breakfast ideas to start your day off right:\n\n**1. Avocado Toast:**\n   - Spread mashed avocado on a slice of toasted bread. Top it with a sprinkle of salt and pepper, a squeeze of lemon or lime juice, and a drizzle of extra virgin olive oil. You can also add sliced tomatoes, chopped onions, or a fried egg on top.\n\n**2. Smoothie Bowl:**\n   - Blend your favorite fruits (such as bananas, strawberries, blueberries, and mango) with a liquid base (like milk, yogurt, or juice) to create a thick and creamy smoothie. Pour it into a bowl and top it with granola, nuts, seeds, shredded coconut, or fresh fruit slices.\n\n**3. Pancakes or Waffles:**\n   - Whip up a batch of fluffy pancakes or waffles and serve them with a variety of toppings. Some options includ

In [8]:
prompt = "I am also vegan. Can you replace the butter too?"
context = user_session.get_recent()
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I am also vegan. Can you replace the butter too?

LLM:  Certainly! For a vegan version of fluffy pancakes, you can replace both the eggs and butter with plant-based alternatives. Here's the recipe tailored for vegan dietary preferences:

**Ingredients:**

- 2 cups (250 grams) all-purpose flour
- 2 teaspoons baking powder
- 1 teaspoon salt
- 2 tablespoons sugar
- 2 cups plant-based milk (such as soy, almond, or oat milk)
- 2 tablespoons neutral-flavored oil (such as canola or vegetable oil), plus more for the pan
- 1 teaspoon vanilla extract (optional)

**Instructions:**

1. Whisk together the dry ingredients: In a large mixing bowl, combine the flour, baking powder, salt, and sugar. Mix well to ensure there are no lumps.

2. Combine the wet ingredients: In a separate mixing bowl, whisk together the plant-based milk, oil, and vanilla extract (if using).

3. Mix dry and wet ingredients: Slowly pour the wet ingredients into the dry ingredients, whisking continuously until you have 

In [9]:
for ctx in context:
    print(ctx)

{'role': 'llm', 'content': "Certainly! Here are some delicious breakfast ideas to start your day off right:\n\n**1. Avocado Toast:**\n   - Spread mashed avocado on a slice of toasted bread. Top it with a sprinkle of salt and pepper, a squeeze of lemon or lime juice, and a drizzle of extra virgin olive oil. You can also add sliced tomatoes, chopped onions, or a fried egg on top.\n\n**2. Smoothie Bowl:**\n   - Blend your favorite fruits (such as bananas, strawberries, blueberries, and mango) with a liquid base (like milk, yogurt, or juice) to create a thick and creamy smoothie. Pour it into a bowl and top it with granola, nuts, seeds, shredded coconut, or fresh fruit slices.\n\n**3. Pancakes or Waffles:**\n   - Whip up a batch of fluffy pancakes or waffles and serve them with a variety of toppings. Some options include fresh fruit, maple syrup, butter, whipped cream, chocolate chips, or a dusting of powdered sugar.\n\n**4. Breakfast Burrito:**\n   - Fill a flour tortilla with scrambled e

This works, but our context keeps growing, increasing our LLM token count, which increases latency and cost.
Conversation histories can be truncated, but that can break quickly.

In [10]:
prompt = "I changed my mind. Can you give me the first recipe from your list?"
context = user_session.get_recent(top_k=5)
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I changed my mind. Can you give me the first recipe from your list?

LLM:  Sure! Here's the recipe for classic fluffy pancakes with eggs and butter:

**Ingredients:**

- 2 cups (250 grams) all-purpose flour
- 2 teaspoons baking powder
- 1 teaspoon salt
- 2 tablespoons sugar
- 2 cups milk
- 2 eggs
- 3 tablespoons melted butter, plus more for the pan
- 1 teaspoon vanilla extract (optional)

**Instructions:**

1. Whisk together the dry ingredients: In a large mixing bowl, combine the flour, baking powder, salt, and sugar. Whisk until the ingredients are well incorporated and there are no lumps.

2. Combine the wet ingredients: In a separate mixing bowl, whisk together the milk, eggs, melted butter, and vanilla extract (if using). Ensure the melted butter has cooled down slightly so it doesn't cook the eggs.

3. Mix dry and wet ingredients: Slowly pour the wet ingredients into the dry ingredients while continuously whisking. Mix until you have a smooth batter, but be careful not to 

In [11]:
for ctx in context:
    print(ctx)

{'role': 'llm', 'content': "Of course! Here is a classic recipe for fluffy and delicious pancakes:\n\n**Ingredients:**\n\n- 2 cups (250 grams) all-purpose flour\n- 2 teaspoons baking powder\n- 1 teaspoon salt\n- 2 tablespoons sugar\n- 2 cups milk\n- 2 eggs\n- 3 tablespoons melted butter, plus more for the pan\n- 1 teaspoon vanilla extract (optional)\n\n**Instructions:**\n\n1. Whisk together the dry ingredients: In a large mixing bowl, combine the flour, baking powder, salt, and sugar. Whisk until well blended.\n\n2. Combine the wet ingredients: In a separate mixing bowl, whisk together the milk, eggs, melted butter, and vanilla extract (if using). Make sure the melted butter has cooled down a bit so it doesn't cook the eggs.\n\n3. Mix dry and wet ingredients: Slowly pour the wet ingredients into the dry ingredients while continuously whisking until you have a smooth batter. Be careful not to overmix; a few small lumps are okay.\n\n4. Heat your griddle or pan: Place a nonstick pan or gr

The LLM has forgotten about the avocado toast it mentioned because that was too far back in it's conversation history!

## Semantic session memory

Rather than naively pass the most recent conversation we can use RedisVL to find only the semantically relevant sections of our conversation based on our new prompt. We can do this with the `fetch_relevant()` method.

In [12]:
prompt = "Can you give me the avocado one?"
user_session.set_distance_threshold(0.75)
context = user_session.get_relevant(prompt=prompt)
response = client.converse(prompt=prompt, context=context)
user_session.store(prompt, response)
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  Can you give me the avocado one?

LLM:  Sure! Here's the first breakfast idea, Avocado Toast, with a simple recipe and some optional toppings:

**Avocado Toast**

**Ingredients:**
- 1 ripe avocado
- 2 slices of bread (preferably a hearty variety like sourdough, rye, or multigrain)
- Extra virgin olive oil
- Lemon or lime juice
- Salt and pepper, to taste
- Optional toppings: sliced tomatoes, chopped onions, chili flakes, or a fried egg

**Instructions:**

1. Toast the bread to your desired level of doneness.
2. While the bread is toasting, prepare the avocado. Cut the avocado in half, remove the pit, and scoop out the flesh into a small bowl.
3. Using a fork, mash the avocado until it reaches your desired consistency. You can leave it chunkier or make it smoother, depending on your preference.
4. Add a drizzle of extra virgin olive oil and a squeeze of lemon or lime juice to the mashed avocado. This will enhance the flavor and prevent the avocado from turning brown too quickly. 

In [13]:
for ctx in context:
    print(ctx)

{'role': 'user', 'content': 'can you give me some ideas for breakfast?'}
{'role': 'llm', 'content': "Certainly! Here are some delicious breakfast ideas to start your day off right:\n\n**1. Avocado Toast:**\n   - Spread mashed avocado on a slice of toasted bread. Top it with a sprinkle of salt and pepper, a squeeze of lemon or lime juice, and a drizzle of extra virgin olive oil. You can also add sliced tomatoes, chopped onions, or a fried egg on top.\n\n**2. Smoothie Bowl:**\n   - Blend your favorite fruits (such as bananas, strawberries, blueberries, and mango) with a liquid base (like milk, yogurt, or juice) to create a thick and creamy smoothie. Pour it into a bowl and top it with granola, nuts, seeds, shredded coconut, or fresh fruit slices.\n\n**3. Pancakes or Waffles:**\n   - Whip up a batch of fluffy pancakes or waffles and serve them with a variety of toppings. Some options include fresh fruit, maple syrup, butter, whipped cream, chocolate chips, or a dusting of powdered sugar.\

Looking at the context we see that only the conversation section that had to do with avocados was retrieved. We don't need to know about pancakes to make avocado toast.

In [14]:
user_session.clear()